In [1]:
import pandas as pd
import warnings
import numpy
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

df_user_bag = pd.read_csv('users_bag_of_words_all.csv')
df_anime_bag = pd.read_csv('anime_bag_of_words_all.csv')

df_anime = pd.read_csv('top_and_bottom_anime.csv')
df_anime = df_anime.drop(columns='Id')
df_anime = df_anime.drop(columns='Rating')
df_anime = df_anime.drop(columns='Demographic')

new = pd.DataFrame()
le = preprocessing.LabelEncoder()
for i in range(df_anime.shape[1]):
    temp = df_anime.iloc[:,i].values.tolist()
    temp = le.fit_transform(temp)
    new[df_anime.columns[i]] = temp

df_true = pd.read_csv('anime_matrix_true.csv')
rows = df_true[['Id']].copy()

In [2]:
#Creates similarity matrix of all anime for all users
count = CountVectorizer()
df_similarity = pd.DataFrame()
df_similarity = df_true[['Id']].copy()
users = df_true.columns[1:]

for i in tqdm(range(len(users))):
    list_of_bag_of_words = [df_user_bag.loc[i]['bag of words']]

    count_matrix = count.fit_transform(df_anime_bag['bag of words'])
    try:
        count_matrix2 = count.transform(list_of_bag_of_words)
    except:
        #if user has an empty bag give them a matix of zeros
        count_matrix2 = count.transform(['a'])

    cosine_sim2 = cosine_similarity(count_matrix, count_matrix2)
    # sim_scores = list(enumerate(cosine_sim2))
    sim_scores = []
    for each in cosine_sim2:
        sim_scores.append(each[0])
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
    df_similarity[users[i]] = sim_scores

# print(sim_scores[1][1][0])

100%|██████████| 9028/9028 [02:01<00:00, 74.14it/s]


In [5]:
import pickle

file = open('similarity_matrix.p', 'wb')
pickle.dump(df_similarity,file)

In [6]:
import pickle

file = open('similarity_matrix.p', 'rb')
df_similarity = pickle.load(file)

In [ ]:
#write code to traverse matrix and find the average sim score for 1's and 0's

In [2]:
#this sets up df_predictions with the same rows as this train/test split
x = new
y = rows
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)

#create cross training split for building the models
X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(x_train, y_train, test_size=0.2, random_state=69)
df_predictions = y_cross_test.copy()

In [ ]:
columns = df_true.columns[1:]
predictions = []

avg = []

x = new
y = df_true[columns[i]]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)

#create cross training split for building the models
X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(x_train, y_train, test_size=0.2, random_state=69)
